In [1]:
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('./data_guane/EUR-USD_procesados_OHLC_5T_2017-6.csv',index_col=0,
                         parse_dates=True, infer_datetime_format=True)

In [3]:
print(data_train.shape)
data_train.head()

(6296, 5)


,open,high,low,bid,ask
2017-06-01 00:05:00,1.12390,1.12393,1.12364,1.12376,1.123796
2017-06-01 00:10:00,1.12376,1.12398,1.12366,1.12394,1.123833
2017-06-01 00:15:00,1.12393,1.12423,1.12392,1.12397,1.124104
2017-06-01 00:20:00,1.12397,1.12400,1.12374,1.12381,1.123908
2017-06-01 00:25:00,1.12381,1.12381,1.12334,1.12339,1.123551


In [4]:
%run ../../code/EUtilities.py
EU = EUtilities()

In [5]:
window = 7
X, y_reg, bt = EU.build_dataset(data_train, window=window, bid_col='bid', binary_target=True, PNL=True)

In [6]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1
2017-06-01 00:35:00,1.12376,1.12394,1.12397,1.12381,1.12339,1.12362,1.12350,-0.000025,-0.000083
2017-06-01 00:40:00,1.12394,1.12397,1.12381,1.12339,1.12362,1.12350,1.12355,-0.000051,-0.000115
2017-06-01 00:45:00,1.12397,1.12381,1.12339,1.12362,1.12350,1.12355,1.12341,-0.000030,-0.000251
2017-06-01 00:50:00,1.12381,1.12339,1.12362,1.12350,1.12355,1.12341,1.12335,-0.000117,0.000010
2017-06-01 00:55:00,1.12339,1.12362,1.12350,1.12355,1.12341,1.12335,1.12345,-0.000051,-0.000027


In [7]:
X['open'] = data_train.filter(X.index,axis=0).open
X['high'] = data_train.filter(X.index,axis=0).high
X['low'] = data_train.filter(X.index,axis=0).low

In [8]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1,open,high,low
2017-06-01 00:35:00,1.12376,1.12394,1.12397,1.12381,1.12339,1.12362,1.12350,-0.000025,-0.000083,1.12362,1.12367,1.12349
2017-06-01 00:40:00,1.12394,1.12397,1.12381,1.12339,1.12362,1.12350,1.12355,-0.000051,-0.000115,1.12350,1.12356,1.12344
2017-06-01 00:45:00,1.12397,1.12381,1.12339,1.12362,1.12350,1.12355,1.12341,-0.000030,-0.000251,1.12355,1.12372,1.12340
2017-06-01 00:50:00,1.12381,1.12339,1.12362,1.12350,1.12355,1.12341,1.12335,-0.000117,0.000010,1.12341,1.12347,1.12333
2017-06-01 00:55:00,1.12339,1.12362,1.12350,1.12355,1.12341,1.12335,1.12345,-0.000051,-0.000027,1.12335,1.12355,1.12334


In [9]:
print('target 0: ', sum(bt==0))
print('target 1: ', sum(bt==1))
print('target 2: ', sum(bt==2))
print('target 3: ', sum(bt==3))

target 0:  2001
target 1:  2288
target 2:  993
target 3:  819


In [10]:
for i in EU.v_split(X,4,1,1):
    print('train: ',i[0][0],i[0][-1],'test: ', i[1][0],i[1][-1])

train:  0 832 test:  833 1113
train:  272 1113 test:  1114 1392
train:  516 1392 test:  1393 1675
train:  550 1675 test:  1676 1921
train:  833 1921 test:  1922 1955
train:  1114 1955 test:  1956 2235
train:  1393 2235 test:  2236 2513
train:  1676 2513 test:  2514 2789
train:  1922 2789 test:  2790 3070
train:  1956 3070 test:  3071 3314
train:  2236 3314 test:  3315 3347
train:  2514 3347 test:  3348 3625
train:  2790 3625 test:  3626 3904
train:  3071 3904 test:  3905 4181
train:  3315 4181 test:  4182 4453
train:  3348 4453 test:  4454 4699
train:  3626 4699 test:  4700 4732
train:  3905 4732 test:  4733 5009
train:  4182 5009 test:  5010 5287
train:  4454 5287 test:  5288 5571
train:  4700 5571 test:  5572 5854
train:  4733 5854 test:  5855 6100


In [11]:
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding
from keras.optimizers import RMSprop, SGD

Using TensorFlow backend.


In [12]:
X_test_PNL_0 = X.PNL_0[833:1113]
X_test_PNL_1 = X.PNL_1[833:1113]

In [13]:
X_train = np.array(X[[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])[:832]

In [14]:
X_train.shape

(832, 10)

In [15]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
y_train = to_categorical(bt[:832],num_classes=4)

In [16]:
X_train.shape, y_train.shape

((832, 1, 10), (832, 4))

### red neuronal recurrente
para empezar probaremos unidades LSTM solo con cuatro días de entrenamiento y probando al quinto día

In [102]:
model = Sequential()

model.add(LSTM(32,input_shape=(X_train.shape[1],X_train.shape[2]),activation='tanh', return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [103]:
def compute_class_weigth(y):
    u = np.unique(y)
    r = {}
    for i in range(len(u)):
        r[i] = sum(u[0]==y)/sum(u[i]==y)
        
    return r

In [104]:
print('label 0: ', sum(bt[:832]==0))
print('label 1: ', sum(bt[:832]==1))
print('label 2: ', sum(bt[:832]==2))
print('label 3: ', sum(bt[:832]==3))

label 0:  280
label 1:  305
label 2:  139
label 3:  108


In [105]:
cw = compute_class_weigth(bt[:832])
cw

{0: 1.0, 1: 0.91803278688524592, 2: 2.014388489208633, 3: 2.5925925925925926}

In [117]:
#model.fit(X_train, y_train, epochs=50, batch_size=1000)
#model.fit(X_train, y_train, epochs=2000, batch_size=200,class_weight={0:1, 1:0.8053, 2:2.275, 3:2.5277})
model.fit(X_train, y_train, epochs=5000, batch_size=200,class_weight=cw)
#model.fit(X_train, y_train, epochs=2000, batch_size=200,class_weight=cw)

In [107]:
X_test = np.array(X[[col for col in X.columns if col!='PNL_0' and col!='PNL_1']])[833:1113]

In [108]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_test = scaler.fit_transform(X_test)
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
y_test = to_categorical(bt[833:1113],num_classes=4)

In [109]:
predict = model.predict_classes(X_test)
np.unique(predict)

 32/280 [==>...........................] - ETA: 2s

array([0, 1, 2, 3])

In [110]:
bt_test = bt[833:1113]

In [111]:
print('label 0: ', sum(bt_test==0))
print('label 1: ', sum(bt_test==1))
print('label 2: ', sum(bt_test==2))
print('label 3: ', sum(bt_test==3))

label 0:  91
label 1:  113
label 2:  40
label 3:  36


In [112]:
print('predict 0: ', sum(predict==0))
print('predict 1: ', sum(predict==1))
print('predict 2: ', sum(predict==2))
print('predict 3: ', sum(predict==3))

predict 0:  76
predict 1:  47
predict 2:  61
predict 3:  96


In [113]:
l_zero, l_one = [],[]
count_ones, count_zeros = 0,0
for i in range(len(predict)):
    if predict[i]==1:
        l_zero.append(count_zeros)
        count_zeros = 0
        count_ones += 1
    else:
        l_one.append(count_ones)
        count_zeros += 1
        count_ones = 0
    
    if i==len(predict)-1:
        l_one.append(count_ones)
        l_zero.append(count_zeros)

print('PNL accumulated: ',sum((predict==1)*X_test_PNL_1 + (predict==0)*X_test_PNL_0))
print('buy mes: ', sum(predict==1)+sum(predict==3))
print('sell mes: ',sum(predict==0)+sum(predict==2))
print('longest buy: ', max(l_one))
print('longest sell: ', max(l_zero))
print('correct target 1: ', np.mean((predict==1)*(bt_test==1)))
print('correct target 0: ', np.mean((predict==0)*(bt_test==0)))
print('correct target 2: ', np.mean((predict==2)*(bt_test==2)))
print('correct target 3: ', np.mean((predict==3)*(bt_test==3)))
print('percision total: ', np.mean(predict==bt_test))

PNL accumulated:  0.00212125114
buy mes:  143
sell mes:  137
longest buy:  4
longest sell:  73
correct target 1:  0.1
correct target 0:  0.0821428571429
correct target 2:  0.0285714285714
correct target 3:  0.0928571428571
percision total:  0.303571428571


#### Conlcusiones
cabe anotar que el error del gradiente disminuye demasiado lento, de hecho se utilizaron 5000 epochs y su disminución del error fue poca, mas sin embargo al pesar las etiquetas la red se comporta de una mejor forma, inclusive haciendo que se genere una ganancia de 0.002121 USD en ese día de comercio, habrá que probar como se comporta en todo un mes.

In [105]:
data_test = pd.read_csv('./data_guane/EUR-USD_procesados_OHLC_1T_2017-6.csv',index_col=0,
                         parse_dates=True, infer_datetime_format=True)

In [106]:
data_test.head()

,open,high,low,bid,ask
2017-06-01 00:01:00,1.12390,1.12393,1.12376,1.12382,1.123867
2017-06-01 00:02:00,1.12382,1.12383,1.12371,1.12381,1.123807
2017-06-01 00:03:00,1.12381,1.12381,1.12366,1.12377,1.123775
2017-06-01 00:04:00,1.12377,1.12382,1.12365,1.12366,1.123780
2017-06-01 00:05:00,1.12366,1.12377,1.12364,1.12376,1.123726


In [107]:
window = 100
X_test, y_test_reg, bt_test = EU.build_dataset(data_test, window=window, bid_col='bid', binary_target=True, PNL=True)

In [108]:
print('target 0: ', sum(bt_test==0))
print('target 1: ', sum(bt_test==1))
print('target 2: ', sum(bt_test==2))
print('target 3: ', sum(bt_test==3))

target 0:  7469
target 1:  9312
target 2:  7025
target 3:  5006


In [109]:
X_test['open'] = data_test.filter(X_test.index,axis=0).open
X_test['high'] = data_test.filter(X_test.index,axis=0).high
X_test['low'] = data_test.filter(X_test.index,axis=0).low

X_test_PNL_0 = X_test.PNL_0
X_test_PNL_1 = X_test.PNL_1

X_test = np.array(X_test[[col for col in X_test.columns if col!='PNL_0' and col!='PNL_1']])

scaler = MinMaxScaler(feature_range=(0, 1))
X_test = scaler.fit_transform(X_test)
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
y_test = to_categorical(bt_test,num_classes=4)

X_test.shape, bt_test.shape

((28812, 1, 103), (28812,))

In [112]:
predict = model.predict_classes(X_test)
np.unique(predict)

28160/28812 [============================>.] - ETA: 0s

array([1])

In [113]:
print('predict 0: ', sum(predict==0))
print('predict 1: ', sum(predict==1))
print('predict 2: ', sum(predict==2))
print('predict 3: ', sum(predict==3))

predict 0:  0
predict 1:  28812
predict 2:  0
predict 3:  0


In [114]:
PNL = sum((predict==1)*X_test_PNL_1 + (predict==0)*X_test_PNL_0)
print('PNL final en train: ', PNL)

PNL final en train:  -0.99906622587


In [26]:
l_zero, l_one = [],[]
count_ones, count_zeros = 0,0
for i in range(len(predict)):
    if predict[i]==1:
        l_zero.append(count_zeros)
        count_zeros = 0
        count_ones += 1
    else:
        l_one.append(count_ones)
        count_zeros += 1
        count_ones = 0
    
    if i==len(predict)-1:
        l_one.append(count_ones)
        l_zero.append(count_zeros)

print('buy mes: ', sum(predict==1)+sum(predict==3))
print('sell mes: ',sum(predict==0)+sum(predict==2))
print('longest buy: ', max(l_one))
print('longest sell: ', max(l_zero))
print('correct target 1: ', np.mean((predict==1)*(bt_test==1)))
print('correct target 0: ', np.mean((predict==0)*(bt_test==0)))
print('correct target 2: ', np.mean((predict==2)*(bt_test==2)))
print('correct target 3: ', np.mean((predict==3)*(bt_test==3)))
print('percision total: ', np.mean(predict==bt_test))

buy mes:  28328
sell mes:  484
longest buy:  10372
longest sell:  88
correct target 1:  0.316916562543
correct target 0:  0.00607385811467
correct target 2:  0.0
correct target 3:  0.0
percision total:  0.322990420658


In [80]:
-0.0013*20

-0.026

In [250]:
sum(predict==1)

907